# scrapy_读书网数据入库和链接跟进

在linux系统终端中进入数据库，创建一个和刚才爬取的数据一样结构的表

In [ ]:
# 先连到自己的mysql
mysql -uroot -p

# 创建数据库
create database spider01 charset=utf8;

# 使用数据库
use spider01;

# 创建表
create table book(
    id int primary key auto_increment,
    name varchar(128),
    src varchar(128);
);

# 查看表
select * from book;
# > Empty set (0.01 sec)

In [ ]:
# 查看linux ip
ifconfig

In [ ]:
# 在settings.py中配置数据库信息

# scrapy_readbook_051/scrapy_readbook_051/settings.py

# 随便找个空的地方，添加如下配置

# 参数有两个坑：一个端口号，一个字符集
DB_HOST = '192.168.0.2' # mysql数据库服务器的ip
DB_PORT = 3306 # mysql数据库服务器的端口 端口号是整数
DB_USER = 'root' 
DB_PASSWORD = '123456'
DB_NAME = 'spider01' # 数据库名
# 注意：utf-8的杠-不允许写
DB_CHARSET = 'utf8' # 数据库编码

In [ ]:
# scrapy_readbook_051/scrapy_readbook_051/pipelines.py
# 在pipelines.py中添加自己的管道

class MysqlPipeline:

    def process_item(self, item, spider):
        self.fp.write(str(item))
        
        return item

In [ ]:
# 在settings.py中配置管道
# scrapy_readbook_051/scrapy_readbook_051/settings.py


# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
   "scrapy_readbook_051.pipelines.ScrapyReadbook051Pipeline": 300,
   # MysqlPipeline
   "scrapy_readbook_051.pipelines.MysqlPipeline": 301,
}

In [ ]:
# 自定义编写MysqlPipeline
# scrapy_readbook_051/scrapy_readbook_051/pipelines.py


# 加载settings.py文件
from scrapy.utils.project import get_project_settings
# 链接sql需要的包
import pymysql

class MysqlPipeline:

    def open_spider(self, spider):
        settings = get_project_settings()
        # DB_HOST = '192.168.0.2' # mysql数据库服务器的ip
        # # 端口号是整数
        # DB_PORT = 3306 # mysql数据库服务器的端口
        # DB_USER = 'root' 
        # DB_PASSWORD = '123456'
        # DB_NAME = 'spider01' # 数据库名
        # DB_CHARSET = 'utf8' # 数据库编码
        self.host = settings['DB_HOST']
        self.port = settings['DB_PORT']
        self.user = settings['DB_USER']
        self.password = settings['DB_PASSWORD']
        self.name = settings['DB_NAME']
        self.charset = settings['DB_CHARSET']
        
        self.connect()

    # 需要本地安装pymysql-->pip install pymysql
    def connect(self):
        self.conn = pymysql.connect(
            host=self.host,
            port=self.port,
            user=self.user,
            password=self.password,
            db=self.name,
            charset=self.charset
        )
        self.cursor = self.conn.cursor()


    def process_item(self, item, spider):
        sql = 'insert into book(name, src) values("{}", "{}")'.format(item['name'], item['src'])
        # 执行sql语句
        self.cursor.execute(sql)
        # 提交
        self.conn.commit()

        return item
    

    def close_spider(self, spider):
        self.cursor.close()
        self.conn.close() 

In [ ]:
# 现在可以运行，然后上数据库服务器上查数据

mysql> select * from book;

注意：如果之前`scrapy_readbook_051/scrapy_readbook_051/spiders/read.py`中

```python
class ReadSpider(CrawlSpider):
    name = "read"
    allowed_domains = ["www.dushu.com"]
    start_urls = ["https://www.dushu.com/book/1188.html"]

    # 在本例中follow=False只爬到了前13页的数据
    # follow=True可以爬取所有的数据
    rules = (Rule(LinkExtractor(allow=r"/book/1188_\d+\.html"), callback="parse_item", follow=True),)

    def parse_item(self, response):
        img_list = response.xpath('//div[@class="bookslist"]//img')

        for img in img_list:
            name = img.xpath('./@alt').extract_first()
            src = img.xpath('./@data-original').extract_first()
            
            book = ScrapyReadbook051Item(name=name, src=src)
            yield book

```
follow=False只爬到了前13页的数据

follow=True可以爬取所有的数据
